# Signal to Noise Ratio (SNR)

The SNR in MRI can be decomposed into dependencies on three factors:

1. Voxel volume
2. Data acquisition time
3. Transverse magnetization

## Voxel Volume

The MRI signal is an integral over a volume of the transverse magnetization.  Therefore there is a linear dependency on the MRI signal and the volume of magnetization being examined.  So we have

$$SNR \propto \mathrm{Voxel\ Volume} = \Delta x\ \Delta y\ \Delta z$$

## Data Acquisition Time

Every additional measurement allows for signal averaging since the noise varies at each measurement.  Note that MRI noise is zero-mean with a Gaussian distribution.  Additional data acquisition time increases signal linearly, while the total noise increases as the square root.  So

$$SNR \propto \frac{S}{\sigma} = \frac{T_{acq}}{\sqrt{T_{acq}}} = \sqrt{T_{acq}}$$

where $T_{acq}$ is the _total data acquisition time_ including all TRs and signal averaging, but not including the sequence downtime.

## Transverse Magnetization

The MRI signal is also proportional to the transverse magnetization, including modulation that occurs due to pulse sequence parameters such as TE, TR, flip angle, spin-echo versus gradient-echo, etc.  This is then also varying depending on the MR properties such as proton density, relaxation rates, diffusion coefficients, etc.  Since this varies widely by pulse sequence, we summarize this as

$$SNR \propto f_{seq}$$

For example, in a simple spoiled gradient-echo pulse sequence, 

$$f_{seq} = M_0 \sin(\theta) \frac{1- \exp(-TR/T_1)}{1- \cos(\theta) \exp(-TR/T_1)}$$

## SNR Equation

Putting it all together,

$$SNR \propto f_{seq}\ \mathrm{Voxel\ Volume}\ \sqrt{T_{acq}}$$

## SNR Efficiency

The SNR efficiency for MRI provides a measure of overall efficiency within a MRI scan, which is the SNR normalized by the *total scan time*, $T_{scan}$

$$SNR_{efficiency} = \frac{SNR}{\sqrt{T_{scan}}} \propto f_{seq}\ \mathrm{Voxel\ Volume}\ \sqrt{\frac{T_{acq} }{T_{scan}}}

## SNR efficiency versus TR

The SNR efficiency is useful to examine the choice of TR.  It is especially interesting for spoiled gradient-echo sequences where when we can choose to use the optimal flip angle ("Ernst angle") based on a $T_1$ value of interest.  For these sequences, the optimal flip angle is

$$\theta_{optimal} = \cos^{-1} \exp(-TR/T_1)$$

And $f_{seq}$ is defined above.

Under this condition, we see the following intriguing result ...

In [ ]:
% setup MRI-education-resources path and requirements
cd ../
startup

loading image
loading signal


In [ ]:
TR = logspace(5,5000);
T1 = 800;
theta = arccos(exp(-TR/T1));

% To remove TE or T2 effect
TE = 0; T2 = Inf;
M0 = 1;

SNR_efficiency = MRsignal_spoiled_gradient_echo(theta, TE, TR, M0, T1, T2) ./ sqrt(TR);
subplot(121)
semilogx(TR, theta*180/pi)
xlabel('TR [ms]'), ylabel('Optimal (Ernst) angle [degrees]')

semilogx(TR, SNR_efficiency)
legend(['T_1 = ' int2str(T1) ' ms'])
xlabel('TR [ms]'), ylabel('SNR efficiency')
title('Spoiled gradient echo SNR efficiency')

What this result shows is that, when we choose the optimal flip angle, there is no difference in SNR efficiency between long and short TR sequences!

Since the optimal flip angle converges to 90-degrees at long TR, this result can be applied to spin echo sequences using 90-degree flip angles and long TRs.


## SNR and $T_2^*$ decay

The SNR equation alone implies that the readout time should be as long as possible, as this leads to more total measurement time, but practically this is limited by the $T_2^*$ decay rate, since at some point there will no longer be any signal to measure.  To incorporate the effect of this decay, we need to account for the signal modulations during the readout by integrating the signal area, accounting for $T_2^*$, which will offset signal gains from extending the readout

$$SNR \propto \frac{ \int_0^{T_{read}} \exp(-\tau / T_2^*) d\tau } { \sqrt{T_{read}} }$$

$T_{read}$ is the readout duration, the numerator the signal acquired during this readout time, and the demoninator captures how the total noise scales with the square root of measurement time.  

The above equation applieds for a one-sided decay, as in a gradient echo, while the following applies in the case of two-sided T2* decay, as in a spin echo:
$$SNR \propto \frac{\int_{-T_{read}/2}^{T_{read}/2} | \exp(-\tau / T_2^*)  | d\tau }{\sqrt{T_{read}}}$$

In [ ]:
Tread = linspace(0.1, 10);
T2s = 1;

SNR_T2s_onesided = T2s * (1 - exp(-Tread/Tts)) ./ sqrt(Tread);

SNR_T2s_twosided = T2s * 2 * (1 - exp(-Tread/2 / Tts)) ./ sqrt(Tread);


plot(Tread, SNR_T2s_onesided)
xlabel('T_{read} / T_2^*')
ylable('Relative SNR')

From this result, we can see that the optimal choice of $T_{read}$ from a SNR point of view is approximately

However, this does not take into account the effects of resolution loss that will typically occur due to decaying $T_2^*$ weighting in k-space.